In [1]:
from pyspark.sql import SparkSession

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.83:7077") \
        .appName("Mikaela_Project")\
        .config("spark.dynamicAllocation.enabled", False)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores", 1)\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

In [2]:
%%time
df_02 = spark_session.read.json("hdfs://192.168.2.83:9000/user/ubuntu/input/RC_2009-10.bz2")

CPU times: user 8.5 ms, sys: 1.27 ms, total: 9.76 ms
Wall time: 29.1 s


In [2]:
%%time
df_03 = spark_session.read.json("hdfs://192.168.2.83:9000/user/ubuntu/input/RC_2011-06.bz2")

CPU times: user 21.4 ms, sys: 9.12 ms, total: 30.5 ms
Wall time: 2min 5s


In [3]:
%%time
df_04 = spark_session.read.json("hdfs://192.168.2.83:9000/user/ubuntu/input/RC_2011-07.bz2")

CPU times: user 23.4 ms, sys: 9.18 ms, total: 32.5 ms
Wall time: 2min 6s


In [2]:
%%time
df_2017_01 = spark_session.read.json("hdfs://192.168.2.83:9000/user/ubuntu/input/RC_2017-01.bz2")

CPU times: user 90.6 ms, sys: 21.5 ms, total: 112 ms
Wall time: 8min 33s


In [3]:
import time

In [4]:
time_init = time.time()

In [5]:
%%time
df = spark_session.read.json("hdfs://192.168.2.83:9000/user/ubuntu/input/RC_2011-06.bz2")

CPU times: user 11.2 ms, sys: 11.7 ms, total: 22.9 ms
Wall time: 1min 28s


In [6]:
df.show()

+--------+----------------+----------------------+--------------------+--------------------+----------------+-----------+-------------+-----+------+------+-------+--------+----------+----------+------------+-----+------------+--------------------+------------+---+
|archived|          author|author_flair_css_class|   author_flair_text|                body|controversiality|created_utc|distinguished|downs|edited|gilded|     id| link_id|      name| parent_id|retrieved_on|score|score_hidden|           subreddit|subreddit_id|ups|
+--------+----------------+----------------------+--------------------+--------------------+----------------+-----------+-------------+-----+------+------+-------+--------+----------+----------+------------+-----+------------+--------------------+------------+---+
|    true|       [deleted]|                  null|                null|           [deleted]|               0| 1306886400|         null|    0| false|     0|c1x2mqs|t3_hoiup|t1_c1x2mqs|  t3_hoiup|  142719944

In [7]:
df.printSchema()

root
 |-- archived: boolean (nullable = true)
 |-- author: string (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- body: string (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- created_utc: string (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- downs: long (nullable = true)
 |-- edited: string (nullable = true)
 |-- gilded: long (nullable = true)
 |-- id: string (nullable = true)
 |-- link_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- retrieved_on: long (nullable = true)
 |-- score: long (nullable = true)
 |-- score_hidden: boolean (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)
 |-- ups: long (nullable = true)



In [8]:
df.count()

9766511

In [4]:
%%time
reddit_rdd_2017_01 = df_05.select("body").rdd.flatMap(lambda x: x)
reddit_rdd_2017_01.take(5)

CPU times: user 12.4 ms, sys: 0 ns, total: 12.4 ms
Wall time: 995 ms


In [9]:
reddit_rdd = df.select("body").rdd.flatMap(lambda x: x)
reddit_rdd.take(10)

['[deleted]',
 "It thought the same thing, searched hard when it seemed like it wasn't going anywhere and now I have four interviews in the next two weeks!\n\nKeep on keepin' on broseph!",
 "He has tons of counters in the laning phase:\n\n- Vindicator\n- Demented Shaman\n- Pharaoh\n- Silhouette (but honestly, who does she not counter?)\n- Jeraziah\n- Tundra\n- Torturer\n- Succubus\n\netc.\n\nLater it's just hard disables and good initiation, Hellflower is incredibly good at shutting him down, any true disabler too.",
 "I know exactly how you feel man.  It's amazing how entry-level positions now require 2-3 years experience and a couple of internships aren't enough.",
 "&gt;Then play soldier or scout.\n\nCan't take on a heavy/medic.",
 "Hmm...that's quite a catch of a man you have there. Well done...could you possibly name your house The USS Enterprise or would that be copyright infringement? =P",
 "Ironically I get a lot of my 'major' Euro news from Al-Jazeera.\n\nAfter that it's all l

In [10]:
reddit_rdd2 = reddit_rdd.filter(lambda row: row != "[deleted]")
reddit_rdd2.take(10)

["It thought the same thing, searched hard when it seemed like it wasn't going anywhere and now I have four interviews in the next two weeks!\n\nKeep on keepin' on broseph!",
 "He has tons of counters in the laning phase:\n\n- Vindicator\n- Demented Shaman\n- Pharaoh\n- Silhouette (but honestly, who does she not counter?)\n- Jeraziah\n- Tundra\n- Torturer\n- Succubus\n\netc.\n\nLater it's just hard disables and good initiation, Hellflower is incredibly good at shutting him down, any true disabler too.",
 "I know exactly how you feel man.  It's amazing how entry-level positions now require 2-3 years experience and a couple of internships aren't enough.",
 "&gt;Then play soldier or scout.\n\nCan't take on a heavy/medic.",
 "Hmm...that's quite a catch of a man you have there. Well done...could you possibly name your house The USS Enterprise or would that be copyright infringement? =P",
 "Ironically I get a lot of my 'major' Euro news from Al-Jazeera.\n\nAfter that it's all local or region

In [11]:
import re
def lower_rdd(lines):
      lines = lines.lower()
      words = re.findall(r'[a-zA-Z]+', lines)
      return words
reddit_words = reddit_rdd2.map(lower_rdd)
reddit_words.take(10)

[['it',
  'thought',
  'the',
  'same',
  'thing',
  'searched',
  'hard',
  'when',
  'it',
  'seemed',
  'like',
  'it',
  'wasn',
  't',
  'going',
  'anywhere',
  'and',
  'now',
  'i',
  'have',
  'four',
  'interviews',
  'in',
  'the',
  'next',
  'two',
  'weeks',
  'keep',
  'on',
  'keepin',
  'on',
  'broseph'],
 ['he',
  'has',
  'tons',
  'of',
  'counters',
  'in',
  'the',
  'laning',
  'phase',
  'vindicator',
  'demented',
  'shaman',
  'pharaoh',
  'silhouette',
  'but',
  'honestly',
  'who',
  'does',
  'she',
  'not',
  'counter',
  'jeraziah',
  'tundra',
  'torturer',
  'succubus',
  'etc',
  'later',
  'it',
  's',
  'just',
  'hard',
  'disables',
  'and',
  'good',
  'initiation',
  'hellflower',
  'is',
  'incredibly',
  'good',
  'at',
  'shutting',
  'him',
  'down',
  'any',
  'true',
  'disabler',
  'too'],
 ['i',
  'know',
  'exactly',
  'how',
  'you',
  'feel',
  'man',
  'it',
  's',
  'amazing',
  'how',
  'entry',
  'level',
  'positions',
  'now',


In [15]:
from nltk.corpus import stopwords
stopword_list = stopwords.words('english')
stopword_list.append('http')
stopword_list.append('com')

In [16]:
def removeStopWordsFunct(x):
    filteredSentence = [w for w in x if not w in stopword_list]
    return filteredSentence
stopword_reddit = reddit_words.map(removeStopWordsFunct)

In [17]:
stopword_reddit.take(10)

[['thought',
  'thing',
  'searched',
  'hard',
  'seemed',
  'like',
  'going',
  'anywhere',
  'four',
  'interviews',
  'next',
  'two',
  'weeks',
  'keep',
  'keepin',
  'broseph'],
 ['tons',
  'counters',
  'laning',
  'phase',
  'vindicator',
  'demented',
  'shaman',
  'pharaoh',
  'silhouette',
  'honestly',
  'counter',
  'jeraziah',
  'tundra',
  'torturer',
  'succubus',
  'etc',
  'later',
  'hard',
  'disables',
  'good',
  'initiation',
  'hellflower',
  'incredibly',
  'good',
  'shutting',
  'true',
  'disabler'],
 ['know',
  'exactly',
  'feel',
  'man',
  'amazing',
  'entry',
  'level',
  'positions',
  'require',
  'years',
  'experience',
  'couple',
  'internships',
  'enough'],
 ['gt', 'play', 'soldier', 'scout', 'take', 'heavy', 'medic'],
 ['hmm',
  'quite',
  'catch',
  'man',
  'well',
  'done',
  'could',
  'possibly',
  'name',
  'house',
  'uss',
  'enterprise',
  'would',
  'copyright',
  'infringement',
  'p'],
 ['ironically',
  'get',
  'lot',
  'major'

In [18]:
def count_words(lines):
    counts = lines.flatMap(lambda line: line) \
        .map(lambda word: (word, 1)) \
        .reduceByKey(lambda a, b: a + b) 
    return counts.sortBy(lambda a: a[1], False)

In [19]:
word_counts = count_words(stopword_reddit)
word_counts.collect()

[('like', 1336082),
 ('would', 1007222),
 ('one', 972899),
 ('people', 968006),
 ('get', 919777),
 ('think', 770083),
 ('know', 635489),
 ('time', 630918),
 ('really', 615540),
 ('good', 588170),
 ('gt', 513810),
 ('also', 496955),
 ('much', 494461),
 ('make', 487558),
 ('even', 480145),
 ('see', 463930),
 ('go', 454549),
 ('well', 453212),
 ('way', 452084),
 ('want', 445451),
 ('www', 433445),
 ('could', 427470),
 ('going', 394079),
 ('something', 390411),
 ('right', 390294),
 ('say', 377506),
 ('still', 375270),
 ('never', 350001),
 ('first', 328446),
 ('got', 323671),
 ('thing', 321154),
 ('actually', 320406),
 ('work', 318130),
 ('amp', 317455),
 ('back', 317261),
 ('need', 309452),
 ('use', 306063),
 ('things', 304527),
 ('pretty', 303039),
 ('though', 298917),
 ('sure', 296629),
 ('better', 288585),
 ('take', 288355),
 ('someone', 286832),
 ('lot', 280337),
 ('us', 279557),
 ('game', 270954),
 ('said', 256334),
 ('many', 253553),
 ('day', 250207),
 ('years', 250041),
 ('every', 2

In [13]:
time_final = time.time()
time_final-time_init

258.3187446594238

In [12]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> l
Packages:
  [ ] pe08................ Cross-Framework and Cross-Domain Parser
                           Evaluation Shared Task

Collections:

([*] marks installed packages)

Download which package (l=list; x=cancel)?
  Identifier> all


       | 
       | Downloading package abc to /home/ubuntu/nltk_data...
       |   Package abc is already up-to-date!
       | Downloading package alpino to /home/ubuntu/nltk_data...
       |   Package alpino is already up-to-date!
       | Downloading package biocreative_ppi to
       |     /home/ubuntu/nltk_data...
       |   Package biocreative_ppi is already up-to-date!
       | Downloading package brown to /home/ubuntu/nltk_data...
       |   Package brown is already up-to-date!
       | Downloading package brown_tei to /home/ubuntu/nltk_data...
       |   Package brown_tei is already up-to-date!
       | Downloading package cess_cat to /home/ubuntu/nltk_data...
       |   Package cess_cat is already up-to-date!
       | Downloading package cess_esp to /home/ubuntu/nltk_data...
       |   Package cess_esp is already up-to-date!
       | Downloading package chat80 to /home/ubuntu/nltk_data...
       |   Package chat80 is already up-to-date!
       | Downloading package city_databas


---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> l
Packages:
  [ ] pe08................ Cross-Framework and Cross-Domain Parser
                           Evaluation Shared Task

Collections:

([*] marks installed packages)

Download which package (l=list; x=cancel)?
  Identifier> nltk.corpus


    Error loading nltk.corpus: Package 'nltk.corpus' not found in
        index



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> nltk all
Command 'nltk all' unrecognized

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> l

Packages:
  [*] abc................. Australian Broadcasting Commission 2006
  [*] alpino.............. Alpino Dutch Treebank
  [*] averaged_perceptron_tagger Averaged Perceptron Tagger
  [*] averaged_perceptron_tagger_ru Averaged Perceptron Tagger (Russian)
  [*] basque_grammars..... Grammars for Basque
  [*] biocreative_ppi..... BioCreAtIvE (Critical Assessment of Information
                           Extraction Systems in Biology)
  [*] bllip_wsj_no_aux.... BLLIP Parser: WSJ 

Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all-nltk


       | 
       | Downloading package abc to /home/ubuntu/nltk_data...
       |   Package abc is already up-to-date!
       | Downloading package alpino to /home/ubuntu/nltk_data...
       |   Package alpino is already up-to-date!
       | Downloading package biocreative_ppi to
       |     /home/ubuntu/nltk_data...
       |   Package biocreative_ppi is already up-to-date!
       | Downloading package brown to /home/ubuntu/nltk_data...
       |   Package brown is already up-to-date!
       | Downloading package brown_tei to /home/ubuntu/nltk_data...
       |   Package brown_tei is already up-to-date!
       | Downloading package cess_cat to /home/ubuntu/nltk_data...
       |   Package cess_cat is already up-to-date!
       | Downloading package cess_esp to /home/ubuntu/nltk_data...
       |   Package cess_esp is already up-to-date!
       | Downloading package chat80 to /home/ubuntu/nltk_data...
       |   Package chat80 is already up-to-date!
       | Downloading package city_databas


---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [5]:
reddit_rdd_02 = df_02.select("body").rdd.flatMap(lambda x: x)
reddit_rdd_02.take(10)

["It's not garbage, its fear mongering.",
 "Actually, it's nice to have a new idea too, especially when you say stuff about how your thing is gonna do for the web what the Mac did for the personal computer. If there was any evidence of that kind of thing to be smelled anywhere, I think you'd find a little more willingness to wait and see on the execution.",
 "I've read them, and they are not that funny. I would suggest you go to bash.org and read top 100, those are much better.",
 "If you look at a lot of these changes, they are really extracted refinements rather than bloating additions. They aim at making things more DRY and eliminating workarounds that used to be more clumsy. So while the Rails API may gain a few methods, it will actually make the programmer's experience using Rails more consistent and streamlined.\r\n\r\nThe beauty of a framework like rails is that you can use it without leveraging its full api - go ahead and write your stuff 'old skool' in 15 lines of code, and a 

In [10]:
reddit_words_02 = reddit_rdd_022.map(lower_rdd)
reddit_words_02.take(10)

[['it', 's', 'not', 'garbage', 'its', 'fear', 'mongering'],
 ['actually',
  'it',
  's',
  'nice',
  'to',
  'have',
  'a',
  'new',
  'idea',
  'too',
  'especially',
  'when',
  'you',
  'say',
  'stuff',
  'about',
  'how',
  'your',
  'thing',
  'is',
  'gonna',
  'do',
  'for',
  'the',
  'web',
  'what',
  'the',
  'mac',
  'did',
  'for',
  'the',
  'personal',
  'computer',
  'if',
  'there',
  'was',
  'any',
  'evidence',
  'of',
  'that',
  'kind',
  'of',
  'thing',
  'to',
  'be',
  'smelled',
  'anywhere',
  'i',
  'think',
  'you',
  'd',
  'find',
  'a',
  'little',
  'more',
  'willingness',
  'to',
  'wait',
  'and',
  'see',
  'on',
  'the',
  'execution'],
 ['i',
  've',
  'read',
  'them',
  'and',
  'they',
  'are',
  'not',
  'that',
  'funny',
  'i',
  'would',
  'suggest',
  'you',
  'go',
  'to',
  'bash',
  'org',
  'and',
  'read',
  'top',
  'those',
  'are',
  'much',
  'better'],
 ['if',
  'you',
  'look',
  'at',
  'a',
  'lot',
  'of',
  'these',
  'cha

In [7]:
reddit_rdd_022 = reddit_rdd_02.filter(lambda row: row != "[deleted]")
reddit_rdd_022.take(10)

["It's not garbage, its fear mongering.",
 "Actually, it's nice to have a new idea too, especially when you say stuff about how your thing is gonna do for the web what the Mac did for the personal computer. If there was any evidence of that kind of thing to be smelled anywhere, I think you'd find a little more willingness to wait and see on the execution.",
 "I've read them, and they are not that funny. I would suggest you go to bash.org and read top 100, those are much better.",
 "If you look at a lot of these changes, they are really extracted refinements rather than bloating additions. They aim at making things more DRY and eliminating workarounds that used to be more clumsy. So while the Rails API may gain a few methods, it will actually make the programmer's experience using Rails more consistent and streamlined.\r\n\r\nThe beauty of a framework like rails is that you can use it without leveraging its full api - go ahead and write your stuff 'old skool' in 15 lines of code, and a 

In [20]:
stopword_reddit_02 = reddit_words_02.map(removeStopWordsFunct)
stopword_reddit_02.take(10)

[['garbage', 'fear', 'mongering'],
 ['actually',
  'nice',
  'new',
  'idea',
  'especially',
  'say',
  'stuff',
  'thing',
  'gonna',
  'web',
  'mac',
  'personal',
  'computer',
  'evidence',
  'kind',
  'thing',
  'smelled',
  'anywhere',
  'think',
  'find',
  'little',
  'willingness',
  'wait',
  'see',
  'execution'],
 ['read',
  'funny',
  'would',
  'suggest',
  'go',
  'bash',
  'org',
  'read',
  'top',
  'much',
  'better'],
 ['look',
  'lot',
  'changes',
  'really',
  'extracted',
  'refinements',
  'rather',
  'bloating',
  'additions',
  'aim',
  'making',
  'things',
  'dry',
  'eliminating',
  'workarounds',
  'used',
  'clumsy',
  'rails',
  'api',
  'may',
  'gain',
  'methods',
  'actually',
  'make',
  'programmer',
  'experience',
  'using',
  'rails',
  'consistent',
  'streamlined',
  'beauty',
  'framework',
  'like',
  'rails',
  'use',
  'without',
  'leveraging',
  'full',
  'api',
  'go',
  'ahead',
  'write',
  'stuff',
  'old',
  'skool',
  'lines',
  

In [23]:
%%time
word_counts_02 = count_words(stopword_reddit_02)
word_counts_02.collect()

CPU times: user 29.3 ms, sys: 4.4 ms, total: 33.7 ms
Wall time: 3.4 s


In [ ]:
# gives a single float value
psutil.cpu_percent()
# gives an object with many fields
psutil.virtual_memory()
# you can convert that object to a dictionary 
dict(psutil.virtual_memory()._asdict())

In [28]:
# release the cores for another application!
spark_context.stop()